# Segmenting and Clustering Neighborhoods in Toronto

In [228]:
import numpy as np, pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

## Part 1: Parsing html data and generating dataframe

In [229]:
d1 = pd.read_html(url)

In [230]:
html_table =  d1[0]
html_table.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Removing areas which do not have Borough assigned to it

In [231]:
html_table = html_table[html_table['Borough']!='Not assigned']
html_table.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


### Replacing 'Not assigned' Neighborhoods with Borough names

In [232]:
for ind in np.where(html_table['Neighbourhood']=='Not assigned'):
    html_table.iloc[ind,2] = html_table.iloc[ind,1]

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Combining Neighborhoods by Postcode

In [236]:
res = []
for postcode in list(set(html_table['Postcode'])):
    postcode_ind = np.where(html_table['Postcode']==postcode)[0]
    if(len(postcode_ind)==1):
        res.append(list(html_table.iloc[postcode_ind,].values[0]))
    else:
        res.append(list(html_table.iloc[postcode_ind[0],:2])+[', '.join(html_table.iloc[postcode_ind,2])])

In [245]:
df = pd.DataFrame(res,columns=html_table.columns,dtype='str')
print('Shape of Resulting Postal Code dataframe:',df.shape)
df.head()

Shape of Resulting Postal Code dataframe: (103, 3)


,Postcode,Borough,Neighbourhood
0,M9W,Etobicoke,Northwest
1,M5S,Downtown Toronto,"Harbord, University of Toronto"
2,M3J,North York,"Northwood Park, York University"
3,M2H,North York,Hillcrest Village
4,M9C,Etobicoke,"Bloordale Gardens, Eringate, Markland Wood, Ol..."


## Part 2: Fetching lat,long values for each Postcode
